Purpose of this notebook, is to take all the docked data and the aligned files and the ligand information and create one large dataframe/csv. other analyisis can be done seperately. 


In [14]:
## import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
# from schrodinger import structure
# from schrodinger.structutils import rmsd
# from schrodinger.structutils import structalign
# from schrodinger.structutils import transform 
# from schrodinger.structutils import sitealign

# from schrodinger.structutils import analyze
import re
# from scipy.spatial import distance
import os
import glob
# from scipy import stats


In [15]:
#UPDATE THESE PATHS!
oak_dir = '/Users/ryan/sherlock_oak/'
data_dir = oak_dir+ "projects/ligand-docking/modeled_structures/dock_min_exp/"

output_path = 'results_af_dock_min_exp.csv'

mcss_types_file = '/Users/ryan/sherlock_home/combind_multistruct_benchmarking/features/mcss16.typ'
SCHRODINGER_dir = ' /opt/schrodinger/suites2022-4/'

# gpcr_db = oak_dir+ "projects/ligand-docking/modeled_structures/scripts/gpcr_db_set.csv"

code to use GPCRDB commented out to be more useful for other datasets, but is probably useful for GPCR

In [16]:
# db = pd.read_csv(gpcr_db)
# db['protein']= db['Unnamed: 1']
# db['struct_name']=db['Unnamed: 7'].apply(lambda x: x if'+' not in str(x) else '6E59' )
# db[db['struct_name']=='6E59']
# len(db[db['Unnamed: 4']!= "C (Glutamate)"]['Unnamed: 1'].unique())
# len(db[db['Unnamed: 4']!= "C (Glutamate)"]['Unnamed: 1'].unique())

#also for GPCRDB
def create_structs_list(protein_db):
    structs = list(protein_db['struct_name'].sort_values())
    if protein == 'MTR1B':
        structs.append('AF_6ME7')
        structs.append('prime_6ME7')
        structs.append('Gpcrdb_active_6ME6')
        structs.append("Gpcrdb_inactive_6ME6")

    else:
        structs.append("AF_"+structs[0])
        structs.append("prime_"+structs[0])
        structs.append('Gpcrdb_active_'+structs[0])
        structs.append('Gpcrdb_inactive_'+structs[0])

    return structs

def create_structs_list(path):
    paths = glob.glob(path+ '/structures/raw/*_prot.mae')
    structs = [l.split("/")[-1].split('_prot.')[0] for l in paths]
    return structs

function to parse RMSD calculated files:

In [17]:
# def parse_rms(rms):
#     m = re.match("(.*\n)+(RMSD: )(\d+.\d+)(.*)",rms[1])
#     if m:
#         return float(m.groups()[2])
#     else:
#         print('failed')
#         return None

In [18]:
#we are going to omit these later:
class_C_proteins =  ['GABR2','CASR', 'GRM2']
skip_no_double_lig = ['']
print("okay here")

okay here


In [19]:
# combind
from tqdm import tqdm

#dictionaries with data:
combind_rmsd_pairs={}
glide_rmsd_pairs={}
glide_rmsd_pairs_top_2={}
glide_rmsd_pairs_top_3={}

glide_rmsd_pairs_top_5={}
glide_rmsd_pairs_top_10={}
glide_rmsd_pairs_top_50={}
glide_rmsd_pairs_top_100={}


min_rmsd_pairs={}
failed_combind_bool = False
failed_before_dock = []
failed_dock = []
failed_combind=[]
top_pose_id = 0
# commented to avoid GPCRDB:
# for protein in db['protein'].unique():
proteins = []
for proteinPath in tqdm(glob.glob(data_dir+"*")):
    #skip the following class C:
    protein = proteinPath.split("/")[-1]

    if protein in class_C_proteins or protein in skip_no_double_lig:
        continue
    if not os.path.exists(data_dir+protein):
        print('where is this folder? '+ protein)
        continue
        
#     protein_db = db[db['protein']==protein]
    #GPCRDB paths:
#     structs = create_structs_list(protein_db)
    #Alt:
    structs = create_structs_list(data_dir+protein)
    if not len(structs) == 0:
        proteins.append(protein)
    
    for struct in structs:
#        ligs = structs  # FOR NON-GLOSA
        ligs = list(filter(lambda x: "old" not in x and "DS_STORE" not in x, os.listdir(data_dir+protein+"/structures/aligned/")))
        print(ligs)
        for ligand_s in ligs:            
            if "_" in ligand_s:
#                 print (ligand_s)
                continue
            pair = ligand_s+"_lig-to-"+struct
            if not os.path.exists(data_dir+protein+'/docking/'+pair+"/"+pair+".log"):
                failed_before_dock.append((protein, pair))
                print("failed before dock: ", (protein,pair))
                continue
            if not os.path.exists(data_dir+protein+'/docking/'+pair+"/"+pair+"_pv.maegz"):
                failed_dock.append((protein,pair))   
                #append 200 so that it's definitely bad!
                combind_rmsd_pairs[(protein,pair)] = 200
                glide_rmsd_pairs[(protein,pair)]= 200
                continue
            failed_combind_bool = False
            if not os.path.exists(data_dir+protein+'/out_'+struct+'.csv'):
                failed_combind.append((protein, struct))
                poses = None
                failed_combind_bool = True
                top_pose_id = 0
            else: 
                poses =pd.read_csv(data_dir+protein+'/out_'+struct+'.csv')
                top_pose_id =  int(poses[poses['ID']==pair]['POSE'])
            if not os.path.exists(data_dir+protein+'/docking/'+pair+"/"+pair+"_pv_rmsd_to.npy"):
                print('no rmsd file' + pair)
                print(protein+'/docking/'+pair+"/"+pair+"_pv_rmsd_to.npy")
                continue
#             else:
#                 print('h')
            rmsds = np.load(data_dir+protein+'/docking/'+pair+"/"+pair+"_pv_rmsd_to.npy")
#             print(rmsds)
            if failed_combind_bool:
                combind_rmsd_pairs[(protein,pair)] = np.nan
            else:
                combind_rmsd_pairs[(protein,pair)] = rmsds[top_pose_id]
            glide_rmsd_pairs[(protein,pair)]=rmsds[0]
            rmsds_len = len(rmsds)
            glide_rmsd_pairs_top_2[(protein,pair)]=min(rmsds[0:min(2, rmsds_len)]) 
            glide_rmsd_pairs_top_3[(protein,pair)]=min(rmsds[0:min(3, rmsds_len)]) 
            glide_rmsd_pairs_top_5[(protein,pair)]=min(rmsds[0:min(5, rmsds_len)]) 
            glide_rmsd_pairs_top_10[(protein,pair)]=min(rmsds[0:min(10, rmsds_len)])
            glide_rmsd_pairs_top_50[(protein,pair)]=min(rmsds[0:min(50, rmsds_len)])
            glide_rmsd_pairs_top_100[(protein,pair)]=min(rmsds[0:min(100, rmsds_len)])
            min_rmsd_pairs[(protein,pair)]=min(rmsds)
print(min_rmsd_pairs)
print(combind_rmsd_pairs)

  5%|██████████▋                                                                                                                                                                                                          | 1/20 [00:00<00:04,  4.68it/s]

['AF_6RZ4', '6RZ5', '6RZ4']


 10%|█████████████████████▎                                                                                                                                                                                               | 2/20 [00:00<00:04,  3.63it/s]

['AF_6AK3', '6AK3', '6M9T']


 15%|███████████████████████████████▉                                                                                                                                                                                     | 3/20 [00:00<00:04,  3.69it/s]

...

['6E59', '6J21', '6HLP', '6J20', 'AF_6E59', '6HLL', '6HLO']
no rmsd file6E59_lig-to-AF_6E59
NK1R/docking/6E59_lig-to-AF_6E59/6E59_lig-to-AF_6E59_pv_rmsd_to.npy


 95%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍          | 19/20 [00:06<00:00,  2.92it/s]

['6K42', '6K41', 'AF_6K41']
['7CX4', '7CX2', '7CX3', 'AF_7CX2']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:06<00:00,  3.04it/s]

{('CLTR1', '6RZ5_lig-to-AF_6RZ4'): 5.0651102037855775, ('CLTR1', '6RZ4_lig-to-AF_6RZ4'): 7.277577653638875, ('PE2R3', '6AK3_lig-to-AF_6AK3'): 1.3222268421990981, ('PE2R3', '6M9T_lig-to-AF_6AK3'): 1.3585363570037865, ('SUCR1', '6Z10_lig-to-AF_6RNK'): 9.63854723491736, ('SUCR1', '6RNK_lig-to-AF_6RNK'): 7.633896951214406, ('PD2R2', '6D27_lig-to-AF_6D26'): 1.9109254286880808, ('PD2R2', '6D26_lig-to-AF_6D26'): 2.5252384438038047, ('DRD1', '7CRH_lig-to-AF_7CKW'): 1.189842725191042, ('DRD1', '7LJC_lig-to-AF_7CKW'): 0.715066569416966, ('DRD1', '7CKZ_lig-to-AF_7CKW'): 0.5314283682415781, ('DRD1', '7JVQ_lig-to-AF_7CKW'): 0.5543265542398724, ('DRD1', '7CKW_lig-to-AF_7CKW'): 1.4971764588029908, ('DRD1', '7JOZ_lig-to-AF_7CKW'): 2.285363023343468, ('DRD1', '7JVP_lig-to-AF_7CKW'): 0.9193042179908036, ('DRD1', '7CKY_lig-to-AF_7CKW'): 7.5848978956365, ('DRD1', '7CKX_lig-to-AF_7CKW'): 0.6615538928648735, ('DRD1', '7JV5_lig-to-AF_7CKW'): 0.9878893433439783, ('DRD1', '7LJD_lig-to-AF_7CKW'): 0.602393628024

In [20]:
import glob
structalign_dict = {}
# data_dir2= oak_dir+ "projects/ligand-docking/modeled_structures/structalign_copy_min/"
#also Relies on GPCRDB here
# for protein in db['protein'].unique():
for protein in tqdm(proteins): 
    #skip the following class C:
    if protein in class_C_proteins or protein in skip_no_double_lig:
        continue
    if not os.path.exists(data_dir+protein):
        print('where is this folder? '+ protein)
        continue
    for alignment_prot in os.listdir(data_dir+protein+"/structures/aligned/"):
        lig_name = alignment_prot+"_lig"
#         print (alignment_prot)
        for output_file in glob.glob(data_dir+protein+"/structures/aligned/"+alignment_prot+"/*to*.out"):
            protein_other = output_file.split('/')[-1].split('_to_')[-1][:-4]
            pair = lig_name+'-to-'+protein_other
            if not os.path.exists(data_dir+protein+'/docking/'+pair):
                continue

            file_align = open(output_file)
            lines = file_align.readlines()
            if len(lines)<1:
                print(lines)
                print('issue with pair '+pair)
                print(protein)

                continue
            last_line = lines[-1]

            rmsd = last_line.split()

            if protein_other == alignment_prot:
                structalign_dict[(protein,pair)]=0.0
                continue
            if len(rmsd)<3 or rmsd[0]!="RMSD:":
                print('issue with'+pair)
                print(protein)
                continue
            rmsd = float(rmsd[-2])
            structalign_dict[(protein,pair)]=rmsd

 84%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                 | 16/19 [00:03<00:00,  4.76it/s]

[]
issue with pair 6E59_lig-to-AF_6E59
NK1R


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:04<00:00,  4.51it/s]


In [21]:
dfs = []
for k in [(glide_rmsd_pairs, 'glide_rmsd'), (glide_rmsd_pairs_top_2, 'glide_rmsd_top_2'),(glide_rmsd_pairs_top_3, 'glide_rmsd_top_3'),(glide_rmsd_pairs_top_5, 'glide_rmsd_top_5'),(glide_rmsd_pairs_top_10, 'glide_rmsd_top_10'), (glide_rmsd_pairs_top_50, 'glide_rmsd_top_50'),(glide_rmsd_pairs_top_100, 'glide_rmsd_top_100'), (min_rmsd_pairs, 'min_pose_rmsd'), (structalign_dict, 'structalign_rmsd'), (combind_rmsd_pairs, 'combind_rmsd')]:
    df = pd.DataFrame.from_dict(k[0], orient = 'index', columns = [k[1]])
    # rmsd_df = rmsd_df.rename(columns = {rmsd_df.index:'pair_name'})
    df['pair_name'] = df.index
    dfs.append(df)
rmsd_df = dfs[0].merge(dfs[1], on = 'pair_name', how = 'outer')
for k in range(2, len(dfs)):
    rmsd_df = rmsd_df.merge(dfs[k], on = 'pair_name', how = 'outer')
rmsd_df['lig'] = rmsd_df['pair_name'].apply(lambda x: x[1].split('-to-')[0]) 
rmsd_df['struct'] = rmsd_df['pair_name'].apply(lambda x: x[1].split('-to-')[-1]) 
rmsd_df['protein'] = rmsd_df['pair_name'].apply(lambda x: x[0])
rmsd_df['pair']= rmsd_df['pair_name'].apply(lambda x: x[1])
rmsd_df.index = rmsd_df['pair_name']


In [22]:
print("total no. pairs")
print(len(glide_rmsd_pairs))
from tabulate import tabulate

methods = ["AF", "Gpcrdb_inactive", "Gpcrdb_active", "prime"]
table = {}

for method in methods:
#    print(f"\n% total {method}")
    pmethod = method
    result = []

    total = len(combind_rmsd_pairs)
    total_af = len([struct for prot, struct in combind_rmsd_pairs if method in struct])
    result.append(total_af / total)

#    print(f"\n% failed {method} before combind")
    total_failed_combind = len(failed_combind)
    total_failed_combind_af = len([struct for prot, struct in failed_combind if method in struct])
    result.append(total_failed_combind_af / total_failed_combind)

#    print(f"\n% failed {method} before dock")
    total_failed_before_dock = len(failed_before_dock)
    total_failed_before_dock_af = len([struct for prot, struct in failed_before_dock if method in struct])
    result.append(total_failed_before_dock_af / total_failed_before_dock)

 #   print(f"\n% failed {method} dock")
    total_failed_dock = len(failed_dock)
    total_failed_dock_af = len([struct for prot, struct in failed_dock if method in struct])
    result.append(total_failed_dock_af / total_failed_dock)
    
    table[pmethod] = np.round(np.array(result) * 100, 2)

comp_exp = np.zeros(len(next(iter(table.values()))))
for method, vals in table.items():
    for i, val in enumerate(vals):
        comp_exp[i] += val
table["experimental"] = 100 - comp_exp

print('ok')
print(tabulate(table, headers="keys", showindex=["% of total", "% of pre-combind fails", 
                                                 "% of pre-dock fails", "% of failed docks"], tablefmt="pretty"))

total no. pairs
64


ZeroDivisionError: division by zero

In [23]:
#similarity mcss:
import tempfile
import subprocess
import os

from rdkit import Chem
#SCHRODINGER_dir = '/opt/schrodinger2021-3/'
#SCHRODINGER_dir = ' /opt/schrodinger/suites2022-3/'
def compute_mcss(smi1, smi2, mcss_types_file):
    """
    Compute smarts patterns for mcss(s) between two structures.
    """
#     cmd = "/share/software/user/restricted/schrodinger/2021-1/
    cmd = SCHRODINGER_dir+"utilities/canvasMCS -ismi {} -ocsv {} -stop 10 -atomtype C {}"
    with tempfile.TemporaryDirectory() as wd:
        smi = wd+'/temp.smi'
        csv = wd+'/temp.csv'

        with open(smi, 'w') as fp:
            fp.write(f'{smi1} st1\n')
            fp.write(f'{smi2} st2\n')

        r = subprocess.run(cmd.format(os.path.basename(smi), os.path.basename(csv),
                                      os.path.abspath(mcss_types_file)),
                           cwd=wd, shell=True, stderr=subprocess.PIPE)


        # mcss can fail with memory usage error, generally when macrocycles
        # are present. just skip such cases.
        failed = 'memory usage' in str(r.stderr)
        print(r.stderr)
        if len(r.stderr) >2:
            mcss = {'st1': ['C'], 'st2': ['C']}
            return mcss
#         print(r.stder)
#         print(csv)
        if not failed:
            assert os.path.exists(csv)
            mcss = {'st1': [], 'st2': []}
            with open(csv) as fp:
                fp.readline()
                for line in fp:
                    lig = line.strip().split(',')[1]
                    smarts = line.strip().split(',')[-1]
                    mcss[lig] += [smarts]
        else:
            print('mcss failed')
            mcss = {'st1': ['C'], 'st2': ['C']}
    return mcss

def get_mcss_size(smi1, smi2, mcss_types_file):

    mcss = compute_mcss(smi1, smi2, mcss_types_file)
#     print(mcss)
    
    mcss = mcss['st1'][0].split(',')[0]
    mcss_st = Chem.MolFromSmiles(mcss, sanitize=False)
    n_mcss_atoms = mcss_st.GetNumAtoms()
#     print(smi1)
#     print(smi2)
    try:
        n_atoms1 = Chem.MolFromSmiles(smi1).GetNumAtoms()
        n_atoms2 = Chem.MolFromSmiles(smi2).GetNumAtoms()
    except:
        return np.nan
    return n_mcss_atoms / min(n_atoms1, n_atoms2)

In [24]:
# tanimoto coefficients
print("starting mcss calculations...")
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Chem import AllChem
chem_list =[]
# cs = []
for protein in proteins:
    #skip the following class C:
    if protein in ['GABR2','CASR']:
        continue
        
#     if protein =='PE2R3':
#         continue
#     af_temp=[]
#     other_temp = []db2 =
#     self_temp = []
#     print(protein)
    if not os.path.exists(data_dir+protein):
        continue
# #     for pair in os.listdir(data_dir+protein+"/docking/"):
    if not os.path.exists(data_dir+protein):
        continue
# #     for pair in os.listdir(data_dir+protein+"/docking/"):
#     protein_db = db[db['protein']==protein]
#     structs = list(protein_db['struct_name'].sort_values())
    structs = []
    c = pd.read_csv(data_dir+protein+'/raw/'+protein+'_native.smi', sep = ' ')
#     cs.append[c]
#     print(protein)
#     print(protein)
    def getMorganFP(x):
        try :
            val = AllChem.GetMorganFingerprintAsBitVect(x,3,128)
            return val
        except:
            return np.nan

    c = c[c["ID"] != "6M9T_lig_old"]
    c['mol']= c['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))
    c['mol_fp'] = c['mol'].apply(lambda x: getMorganFP(x))
    c['struct_name']=c['ID'].apply(lambda x: "AF_" + x.split('_')[0])

    chem_list.append(c)
#     print(chem_list)

# print(chem_list)
db = rmsd_df[['struct','lig']]
#print(db)
#print(pd.concat(chem_list))
db2 = db.merge(pd.concat(chem_list),left_on = 'struct', right_on = 'struct_name',how = "left")
#print(db2)
# print(db2)
def get_fp(y):
    if 'AF_' in y:
#         print(y.split('_')[-1])
        if len(db2[db2['struct_name']==y.split('_')[-1]]['mol_fp'])<1:
            return np.nan
        return db2[db2['struct_name']==y.split('_')[-1]]['mol_fp'].iloc[0]
    if 'prime_' in y:
        return db2[db2['struct_name']==y.split('_')[-1]]['mol_fp'].iloc[0]
    if 'Gpcrdb_' in y:
#         print ("ho")
        return db2[db2['struct_name']==y.split('_')[-1]]['mol_fp'].iloc[0]
    
    out = db2[db2['struct_name']==y]['mol_fp'].iloc[0]
#     print(out)
    return out
def getSimilarity(pair):
    f1 = get_fp(pair.split('_lig')[0])
    f2 = get_fp(pair.split('-to-')[-1])
    if np.isnan(f1).any() or np.isnan(f2).any():
        return np.nan
#     return DataStructs.DiceSimilarity(f1,f2)
    return DataStructs.FingerprintSimilarity(f1, f2, metric = DataStructs.TanimotoSimilarity)

def get_SMILE(y):
    if 'AF_' in y:
#         print(y.split('_')[-1])
        if len(db2[db2['struct_name']==y.split('_')[-1]]['SMILES'])<1:
            return None
        return db2[db2['struct_name']==y.split('_')[-1]]['SMILES'].iloc[0]
    if 'prime_' in y:
        return db2[db2['struct_name']==y.split('_')[-1]]['SMILES'].iloc[0]
    
    if 'Gpcrdb_' in y:
#         print ("ho")
        return db2[db2['struct_name']==y.split('_')[-1]]['SMILES'].iloc[0]
    
    out = db2[db2['struct_name']==y]['SMILES'].iloc[0]
    return out

    
def getMCSS(pair):
    f1 = get_SMILE("AF_" + pair.split('_lig')[0])
    f2 = get_SMILE(pair.split('-to-')[-1])
    if f1 is None or f2 is None:
        return np.nan
#     return DataStructs.DiceSimilarity(f1,f2)
    return get_mcss_size(f1, f2, mcss_types_file)

def getLigandLength(pair):
    f1 = get_SMILE("AF_" + pair.split('_lig')[0])
    try:
        
        return Chem.MolFromSmiles(f1).GetNumAtoms()
    except:
        return np.nan
    
import rdkit.Chem.Descriptors as Descriptors
    
def getLigandDa(pair):
    f1 = get_SMILE("AF_" + pair.split('_lig')[0])
    if Chem.MolFromSmiles(f1) is None:
        return np.nan
    return Descriptors.ExactMolWt(Chem.MolFromSmiles(f1))

def getIsSame(pair):
    f1 = get_SMILE("AF_" + pair.split('_lig')[0])
    f2 = get_SMILE(pair.split('-to-')[-1])
    if f1 is None or f2 is None:
        return np.nan
    
#     f1 = get_SMILE(pair.split('_lig')[0])
    try:
        
        m1=Chem.MolFromSmiles(f1)
        m2=Chem.MolFromSmiles(f2)
    except:
        return np.nan
    if m1 is None or m2 is None:
        return False
    return Chem.MolToInchi(m1) == Chem.MolToInchi(m2) 
# # DiceSimilarity
# rmsd_df['tanimoto']=rmsd_df['pair'].apply(getSimilarity) 
rmsd_df['mcss']= rmsd_df['pair'].apply(getMCSS)

rmsd_df['mol_same']= rmsd_df['pair'].apply(getIsSame)
# rmsd_df['len_ligand']= rmsd_df['pair'].apply(getLigandLength)
# rmsd_df['lig_Da']= rmsd_df['pair'].apply(getLigandDa)

# rmsd_df['native_ligand']= rmsd_df['pair'].apply(lambda x: get_fp(x.split('-to-')[-1]))
# rmsd_df['tanimoto']=rmsd_df['pair'].apply(lambda x: DataStructs.FingerprintSimilarity(get_fp(x.split('_lig')[0]),get_fp(x.split('-to-')[-1]))) 
print('done')


starting mcss calculations...
done


In [25]:
def get_type(x):
    if str(x)=='nan':
        print('here!')
        return np.nan
    lig = x[1].split("_lig")[0]
    prot = x[1].split("-to-")[-1]
    if "AF_" in prot:
        return 'AF_dock'
    elif 'prime_' in prot:
        return 'prime_dock'
    elif 'Gpcrdb' in prot:
        return 'gpcrdb_dock'
    if lig == prot:
        return 'same_dock'
    else:
        return 'cross_dock'
rmsd_df['type']= rmsd_df['pair_name'].apply(get_type)

In [26]:
print('writing to ' + output_path)
print(rmsd_df.columns)
rmsd_df.to_csv(output_path)


writing to results_af_dock_min_exp.csv
Index(['glide_rmsd', 'pair_name', 'glide_rmsd_top_2', 'glide_rmsd_top_3',
       'glide_rmsd_top_5', 'glide_rmsd_top_10', 'glide_rmsd_top_50',
       'glide_rmsd_top_100', 'min_pose_rmsd', 'structalign_rmsd',
       'combind_rmsd', 'lig', 'struct', 'protein', 'pair', 'mcss', 'mol_same',
       'type'],
      dtype='object')
